In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from statannotations.Annotator import Annotator
import seaborn as sns
import statsmodels
from scipy.stats import ttest_ind
from itertools import combinations
from statsmodels.stats.multitest import multipletests

from scipy.stats import kruskal
import scikit_posthocs as sp


In [ ]:
regionprops_df = pd.read_csv("../../turmoric/jul_3_rot_ogd_regionprops.csv")
regionprops_df = regionprops_df[regionprops_df["treatment"] != "30mR_control"]

In [ ]:
treatments = regionprops_df['treatment'].unique()
treatments

In [ ]:
groups = [group['aspect_ratio'].values for name, group in regionprops_df.groupby('treatment')]
kw_stat, kw_p = kruskal(*groups)
print(f"Kruskal-Wallis p-value: {kw_p:.4e}")



In [ ]:
# Step 2: Pairwise posthoc Dunn’s test
dunn_df = sp.posthoc_dunn(regionprops_df, val_col='aspect_ratio', group_col='treatment', p_adjust=None)


In [ ]:
# Flatten the p-values and correct
pairs = []
raw_pvals = []

treatments = dunn_df.columns
for i in range(len(treatments)):
    for j in range(i+1, len(treatments)):
        g1, g2 = treatments[i], treatments[j]
        pairs.append((g1, g2))
        raw_pvals.append(dunn_df.loc[g1, g2])

# Apply correction (Bonferroni, Holm, etc.)
_, corrected_pvals, _, _ = multipletests(raw_pvals, method='bonferroni')  # or 'bonferroni', 'fdr_bh', etc.


In [ ]:
# Build final result table
results = pd.DataFrame(pairs, columns=['Group1', 'Group2'])
results['p_corrected'] = corrected_pvals
results['significance'] = results['p_corrected'].apply(
    lambda p: 'ns' if p > 0.05 else ('*' if p <= 0.05 else '**' if p <= 0.01 else '***')
)

print(results)

significant_results = results[results['p_corrected'] < 0.05].copy()

In [ ]:
plt.figure(figsize=(16, 12))

ax = sns.boxplot(data=regionprops_df, x='treatment', y='aspect_ratio', palette='Set2', )

# Optional: Add error bars
# ...

# Manual annotation
# Starting y value (above the max of your data)
y_base = regionprops_df['aspect_ratio'].max() * 1.01

# How much space between each annotation line
y_step = regionprops_df['aspect_ratio'].max() * 0.1

for i, row in significant_results.iterrows():
    g1, g2 = row['Group1'], row['Group2']
    x1 = list(regionprops_df['treatment'].unique()).index(g1)
    x2 = list(regionprops_df['treatment'].unique()).index(g2)
    
    # Vertical offset increases with each comparison
    y = y_base + i * y_step
    h = y_step * 0.5  # height of the tick

    # Draw annotation line and text
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='black')
    ax.text((x1 + x2) * 0.5, y + h, row['significance'], ha='center', va='bottom', color='black')


plt.yscale('log')
plt.tight_layout()
plt.show()


In [ ]:
# Create the plot
plt.figure(figsize=(8, 6))
sns.set(style="whitegrid")

# Scatter (beeswarm) plot
#ax = sns.stripplot(data=regionprops_df, x="treatment", y='perimeter', jitter=True, size=1, color=".3", alpha=0.5)
ax = sns.swarmplot(data=regionprops_df, x="treatment", y='perimeter', size=0.5, palette='Set2', alpha=0.5)

# Add mean ± SD bars manually


groups = regionprops_df['treatment'].unique()
group_means = regionprops_df.groupby("treatment")["perimeter"].mean()
group_stds = regionprops_df.groupby("treatment")["perimeter"].std()

# for x, group in enumerate(groups):
#     mean = group_means[group]
#     std = group_stds[group]
    
#     lower = mean - std
#     upper = mean + std

#     # Enforce positive range for log scale
#     lower = max(lower, 1e-1)
    
#     ax.plot([x, x], [lower, upper], color='black', linewidth=2)
#     ax.plot(x, mean, 'o', color='black')


# Generate all pairwise combinations
pairs = list(combinations(groups, 2))

annotator = Annotator(ax, pairs, data=regionprops_df, x='treatment', y='perimeter')
annotator.configure(test='t-test_ind', text_format='star', loc='outside', comparisons_correction="bonferroni")
annotator.apply_and_annotate()


plt.yscale("log")
plt.ylabel("Perimeter (Pixels)")
plt.title("Perimeter by Group")
plt.tight_layout()
plt.show()